In [1]:
import pandas as pd
import numpy as np
import io
from plotnine import *
import re
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
estratos = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4"]
estratos.reverse()

estratosRestritos = ["A1", "A2", "A3", "A4"]
estratosRestritos.reverse()

pesos = dict({"B4": 0.05,
               "B3": 0.10,
               "B2": 0.20,
               "B1": 0.50,
               "A4": 0.625,
               "A3": 0.75, 
               "A2": 0.875,
               "A1": 1.0,
               "C": 0
              })
cat_map = dict(
    [('DISCENTE BACHARELADO', 'GRAD'),
     ('DISCENTE DOUTORADO', 'DISCENTE'),
     ('DISCENTE DOUTORADO PROFISSIONAL', 'DISCENTE'),
     ('DISCENTE MESTRADO', 'DISCENTE'),
     ('DISCENTE MESTRADO PROFISSIONAL', 'DISCENTE'),
     ('DOCENTE COLABORADOR', 'DOCENTE COLABORADOR'),
     ('DOCENTE PERMANENTE', 'DOCENTE PERMANENTE'),
     ('DOCENTE VISITANTE', 'DOCENTE VISITANTE'),
     ('EGRESSO', 'DISCENTE'),
     ('EGRESSO_GERAL_5_ANOS', 'DISCENTE'),
     ('PARTICIPANTE EXTERNO', 'PARTICIPANTE EXTERNO'),
     ('PÓS-DOC', 'PÓS-DOC')
    ]
  )
niveismap = dict({
    "MESTRADO": "M",
    "MESTRADO/DOUTORADO": "MD",
    "MESTRADO PROFISSIONAL": "M",
    "MESTRADO PROFISSIONAL/DOUTORADO PROFISSIONAL": "MD",
    "DOUTORADO": "D"
  })


In [3]:
sheetnames = ['producoes_lista', 'programas', 'docentes', 'discentes', 'tcc']
path = './'
sheets = dict([[sn, pd.read_csv(path + 'N-' + sn + '.csv', error_bad_lines=False)] for sn in sheetnames])

<ipython-input-3-fe9a32fb48f6>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


<ipython-input-3-fe9a32fb48f6>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


<ipython-input-3-fe9a32fb48f6>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


<ipython-input-3-fe9a32fb48f6>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


<ipython-input-3-fe9a32fb48f6>:3: DtypeWarning: Columns (7,32) have mixed types. Specify dtype option on import or set low_memory=False.
<ipython-input-3-fe9a32fb48f6>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


In [14]:
producoes = sheets['producoes_lista']
producoes.columns

Index(['Unnamed: 0', 'Ano base', 'Cod PPG', 'Nome PPG', 'Área Básica',
       'IES Principal Sigla', 'IES Principal Nome', 'IES da Rede Sigla',
       'Status Jurídico', 'UF', 'Região', 'PPG Ano Início', 'Modalidade',
       'Mudança de modalidade no quadriênio?', 'Nota', 'Nível',
       'Ano Início Nível', 'Sofreu migração no quadriênio?',
       'Titulo da Produção', 'Tipo', 'Subtipo', 'Natureza',
       'Campo de Correspondência (PTT)      OU Campo Natureza da Obra (Livros)',
       'País', 'Área de Concentração', 'Linha de Pesquisa', 'Projeto',
       'Estrato', 'ISSN no Qualis ', 'Nome do Periódico no Qualis',
       'Nº de Eventos', 'Ciclo Avaliativo', 'Artigo em Periódico',
       'Demais Tipos/Subtipos', 'Docente', 'Nº Docentes Permanentes',
       'Total de Autores', 'Total  de Docentes', 'Permanente', 'Colaborador',
       'Visitante', 'Total de Discentes', 'Disc Dout', 'Disc Dout Prof',
       'Disc Mestr', 'Disc Mestr Prof', 'Disc Grad / Bach',
       'Total de Part Ext', '

In [67]:
wd = producoes[
    (producoes['Subtipo'] == 'TRABALHO EM ANAIS') & 
    (producoes['Evento'].notnull()) &
    (producoes['Evento'] != 'NOTEVENT') &
    (producoes['Natureza'] == 'TRABALHO COMPLETO')
][['Cod PPG', 'Evento', 'ID_ADD_PRODUCAO_INTELECTUAL']] \
    .groupby(['Cod PPG', 'Evento']) \
    .count().reset_index().rename(columns={'ID_ADD_PRODUCAO_INTELECTUAL': 'Qte'}) \
    .groupby(['Cod PPG']) \
    .apply(lambda x: x.sort_values('Qte', ascending=False).assign(Pos=[i + 1 for i in range(len(x.index))])) \
    .reset_index(drop=True)

res = producoes[['ID_ADD_PRODUCAO_INTELECTUAL', 'Cod PPG', 'Evento']] \
    .merge(wd, on=['Cod PPG', 'Evento'], how='right') \
    .reset_index(drop=True)

res.to_csv('eventos-frequencia.csv')

res

,ID_ADD_PRODUCAO_INTELECTUAL,Cod PPG,Evento,Qte,Pos
0,31277072,11001011072P2,SBCARS,1,1
1,31277073,11001011072P2,SBES,1,2
2,29818324,12001015012P2,SBIE,59,1
3,29818312,12001015012P2,SBIE,59,1
4,29818311,12001015012P2,SBIE,59,1
...,...,...,...,...,...
17912,29111460,53001010098P3,CIbSE,1,80
17913,30705332,53001010098P3,LAWCC,1,81
17914,29336095,53001010098P3,CIT,1,82
17915,30704860,53001010098P3,MSR,1,83


In [66]:
wd = producoes[
    (producoes['Subtipo'] == 'ARTIGO EM PERIÓDICO') & 
    (producoes['Natureza'] == 'TRABALHO COMPLETO')
][['Cod PPG', 'Nome do Periódico no Qualis', 'ID_ADD_PRODUCAO_INTELECTUAL']] \
    .groupby(['Cod PPG', 'Nome do Periódico no Qualis']) \
    .count().reset_index().rename(columns={'ID_ADD_PRODUCAO_INTELECTUAL': 'Qte'}) \
    .groupby(['Cod PPG']) \
    .apply(lambda x: x.sort_values('Qte', ascending=False).assign(Pos=[i + 1 for i in range(len(x.index))])) \
    .reset_index(drop=True)

res = producoes[['ID_ADD_PRODUCAO_INTELECTUAL', 'Cod PPG', 'Nome do Periódico no Qualis']] \
    .merge(wd, on=['Cod PPG', 'Nome do Periódico no Qualis'], how='right') \
    .reset_index(drop=True)

res.to_csv('periodico-frequencia.csv')

res

,ID_ADD_PRODUCAO_INTELECTUAL,Cod PPG,Nome do Periódico no Qualis,Qte,Pos
0,29988596,11001011072P2,IEEE TRANSACTIONS ON SOFTWARE ENGINEERING,1,1
1,31277074,11001011072P2,SOFTWARE: PRACTICE AND EXPERIENCE,1,2
2,29824705,12001015012P2,IEEE ACCESS,13,1
3,29824693,12001015012P2,IEEE ACCESS,13,1
4,29824915,12001015012P2,IEEE ACCESS,13,1
...,...,...,...,...,...
11035,30704646,53001010098P3,JOURNAL OF INFORMATION AND DATA MANAGEMENT - JIDM,1,60
11036,33043085,53001010098P3,AD HOC NETWORKS,1,61
11037,33043234,53001010098P3,JOURNAL OF INTERNET TECHNOLOGY,1,62
11038,33043168,53001010098P3,JOURNAL OF MACHINE LEARNING RESEARCH (ONLINE),1,63


In [64]:
producoes[['ISSN no Qualis ', 'Nome do Periódico no Qualis']]

,ISSN no Qualis,Nome do Periódico no Qualis
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
54416,2073-8994,SYMMETRY
54417,NaN,NaN
54418,NaN,NaN
54419,NaN,NaN


In [63]:
producoes.columns

Index(['Unnamed: 0', 'Ano base', 'Cod PPG', 'Nome PPG', 'Área Básica',
       'IES Principal Sigla', 'IES Principal Nome', 'IES da Rede Sigla',
       'Status Jurídico', 'UF', 'Região', 'PPG Ano Início', 'Modalidade',
       'Mudança de modalidade no quadriênio?', 'Nota', 'Nível',
       'Ano Início Nível', 'Sofreu migração no quadriênio?',
       'Titulo da Produção', 'Tipo', 'Subtipo', 'Natureza',
       'Campo de Correspondência (PTT)      OU Campo Natureza da Obra (Livros)',
       'País', 'Área de Concentração', 'Linha de Pesquisa', 'Projeto',
       'Estrato', 'ISSN no Qualis ', 'Nome do Periódico no Qualis',
       'Nº de Eventos', 'Ciclo Avaliativo', 'Artigo em Periódico',
       'Demais Tipos/Subtipos', 'Docente', 'Nº Docentes Permanentes',
       'Total de Autores', 'Total  de Docentes', 'Permanente', 'Colaborador',
       'Visitante', 'Total de Discentes', 'Disc Dout', 'Disc Dout Prof',
       'Disc Mestr', 'Disc Mestr Prof', 'Disc Grad / Bach',
       'Total de Part Ext', '

In [69]:
tcc = sheets['tcc']

tcc.columns

Index(['Unnamed: 0', 'Ano Base', 'Cod PPG', 'Nome PPG', 'Área Básica',
       'IES Principal Sigla', 'IES Principal Nome', 'IES da Rede Sigla',
       'Status Jurídico', 'UF', 'Região', 'PPG Ano Início', 'Modalidade',
       'Mudança de modalidade no quadriênio?', 'Nota/ conceito', 'Nível',
       'Ano Início Nível', 'Sofreu migração no quadriênio?',
       'Área de concentração (nome)', 'Linha de pesquisa (nome)',
       'Projeto de Pesquisa (nome)', 'Subtipo',
       'Título do Trabalho de Conclusão', 'Palavra Chave', 'Idioma',
       'Url do texto completo', 'Grau acadêmico',
       'Identificador pessoa Discente', 'Nome do Discente',
       'Identificador pessoa orientador', 'Nome do orientador',
       'Categoria do orientador', 'Categoria do docente orientador',
       'Nacionalidade orientador', 'Indicador orientador participou da banca',
       'Total de Membros', ' Total Pós-Doc', 'Nº\nPós-Doc estrangeiros ',
       ' Total Egressos', 'Total Docentes', 'N º\nDocentes estrangei

In [71]:
def sumarizar_autores(prod):
    wd = prod
    wd = wd[
                (wd['Subtipo'].isin(["ARTIGO EM PERIÓDICO", "TRABALHO EM ANAIS"])) &
                (wd['Natureza'] == "TRABALHO COMPLETO")
      ]
    wd = wd[wd['ID_ADD_PRODUCAO_INTELECTUAL'].notnull()]
    wd = wd[['Cod PPG', 'ID_ADD_PRODUCAO_INTELECTUAL', 'NM_AUTORES']]
    idsProd = wd['ID_ADD_PRODUCAO_INTELECTUAL'].values.tolist()
    wdsplt = [str(l).split(' | ') for l in wd['NM_AUTORES'].values]
    cols = ['ID_ADD_PRODUCAO_INTELECTUAL'] + list(range(max([len(x) for x in wdsplt])))
    wdsplt = [[idsProd[i]] + wdsplt[i] for i in range(len(wdsplt))]
    wdsplt = pd.DataFrame(wdsplt, columns=cols)
    wd = wd.merge(wdsplt, on='ID_ADD_PRODUCAO_INTELECTUAL', how='left')
    wd = wd.melt(id_vars=['Cod PPG', 'ID_ADD_PRODUCAO_INTELECTUAL', 'NM_AUTORES'], var_name="Pos", value_name="NomeCat")
    wd = wd[wd["NomeCat"].notnull()]
    wd.loc[:, ['NomeRe']] = wd['NomeCat'].apply(lambda x: re.search('(.*) \((.*)\)', x))
    wd.loc[wd['NomeRe'].isnull(), ['Nome']] = wd.loc[wd['NomeRe'].isnull(), ['NM_AUTORES']]
    wd.loc[wd['NomeRe'].isnull(), ['Cat']] = 'NOCAT'
    wd.loc[wd['NomeRe'].notnull(), ['Nome', 'Cat']] = wd[wd['NomeRe'].notnull()]['NomeRe'].apply(lambda x: x.group(1))
    wd.loc[wd['NomeRe'].notnull(), ['Cat']] = wd[wd['NomeRe'].notnull()]['NomeRe'].apply(lambda x: cat_map[x.group(2)])
    wd = wd.drop(columns=["NM_AUTORES", "NomeCat", "NomeRe"])
    wd['Pos'] = wd['Pos'] + 1
    wd = prod[['Cod PPG', 'Estrato', 'ID_ADD_PRODUCAO_INTELECTUAL', 'Subtipo', 'Natureza', 'Ano base']].merge(wd, on=['Cod PPG', 'ID_ADD_PRODUCAO_INTELECTUAL'], how='right')
    wd = wd.rename(columns={'ID_ADD_PRODUCAO_INTELECTUAL': 'idProd', 'Nome': 'Nome do docente'})

    return wd

sumario_autores = sumarizar_autores(producoes)

In [104]:
incluirPub = ["Tot", "Pub", "Per", "Pri"]
def calc(data):
    wd = data
    wd = wd[['ID_ADD_TCC', 'Cod PPG', 'Nome do Discente']]
    wdpub = wd[["Cod PPG", "Nome do Discente"]] \
        .merge(sumario_autores.rename(columns={"Nome do docente": "Nome do Discente"}), on=["Cod PPG", "Nome do Discente"], how="inner")
    wdpub = wd[["Cod PPG", "Nome do Discente"]].merge(sumario_autores.rename(columns={"Nome do docente": "Nome do Discente"}), on=["Cod PPG", "Nome do Discente"], how="inner")
    wdpub = wdpub[
        (wdpub["Natureza"] == "TRABALHO COMPLETO") &
        (wdpub["Subtipo"].isin(["ARTIGO EM PERIÓDICO", "TRABALHO EM ANAIS"])) &
        (wdpub["Estrato"].isin(estratos))
    ]

    wdaut = wd.merge(wdpub, on=['Cod PPG', 'Nome do Discente'], how='inner')
    wdaut = wdaut[['ID_ADD_TCC', 'Cod PPG', 'Nome do Discente', 'idProd']] \
        .groupby(['ID_ADD_TCC', 'Cod PPG', 'Nome do Discente']).count() \
        .reset_index() \
        .rename(columns={'idProd': 'NumPub'})

    wdpubR = wdpub[wdpub['Estrato'].isin(estratosRestritos)]
    wdautR = wd.merge(wdpubR, on=['Cod PPG', 'Nome do Discente'], how='inner')
    wdautR = wdautR[['ID_ADD_TCC', 'Cod PPG', 'Nome do Discente', 'idProd']] \
        .groupby(['ID_ADD_TCC', 'Cod PPG', 'Nome do Discente']).count() \
        .reset_index() \
        .rename(columns={'idProd': 'NumPubRes'})

    res = wd.merge(wdaut, on=['ID_ADD_TCC', 'Cod PPG', 'Nome do Discente'], how='left').fillna(0)
    res = res.merge(wdautR, on=['ID_ADD_TCC', 'Cod PPG', 'Nome do Discente'], how='left').fillna(0)
    for k in ['ID_ADD_TCC', 'NumPub', 'NumPubRes']:
        res[k] = res[k].astype(int)
    res = res.reset_index(drop=True)
    return res

res = calc(tcc)

res.to_csv('pub-aut.csv', index=False)

In [81]:
tcc.columns

Index(['Unnamed: 0', 'Ano Base', 'Cod PPG', 'Nome PPG', 'Área Básica',
       'IES Principal Sigla', 'IES Principal Nome', 'IES da Rede Sigla',
       'Status Jurídico', 'UF', 'Região', 'PPG Ano Início', 'Modalidade',
       'Mudança de modalidade no quadriênio?', 'Nota/ conceito', 'Nível',
       'Ano Início Nível', 'Sofreu migração no quadriênio?',
       'Área de concentração (nome)', 'Linha de pesquisa (nome)',
       'Projeto de Pesquisa (nome)', 'Subtipo',
       'Título do Trabalho de Conclusão', 'Palavra Chave', 'Idioma',
       'Url do texto completo', 'Grau acadêmico',
       'Identificador pessoa Discente', 'Nome do Discente',
       'Identificador pessoa orientador', 'Nome do orientador',
       'Categoria do orientador', 'Categoria do docente orientador',
       'Nacionalidade orientador', 'Indicador orientador participou da banca',
       'Total de Membros', ' Total Pós-Doc', 'Nº\nPós-Doc estrangeiros ',
       ' Total Egressos', 'Total Docentes', 'N º\nDocentes estrangei